In [1]:
# vim: fdm=indent

#author:joanna ahn
#date: 28/06/23
#content: compress drosophila melanogaster


import os
import numpy as np
import pandas as pd

import anndata
import scanpy as sc

from utils import (
    root_repo_folder,
    output_folder,
    #get_tissue_data_dict,
    fix_annotations,
    get_celltype_order,
    collect_gene_annotations,
    store_compressed_atlas,
    )

species = 'd_melanogaster'
full_atlas_data_folder = root_repo_folder / 'data' / 'full_atlases' / 'drosophila_melanogaster'
anno_fn = root_repo_folder / 'data' / 'gene_annotations' / 'dmel-all-r6.31.gtf.gz'  # see paper methods: https://www.science.org/doi/10.1126/science.abk2432
fn_out = output_folder / f'{species}.h5'

def get_tissue_data_dict(species, full_atlas_data_folder, rename_dict):
    result = []
    filenames = os.listdir(full_atlas_data_folder)
    fns = [x for x in filenames if '.h5ad' in x]

    for filename in fns:
        tissue_start = filename.find('biohub_') + len("biohub_")
        tissue_end = filename.rfind('_')
        tissue = filename [tissue_start:tissue_end]
        # TODO: rescue cell types that are found only in "body"
        if tissue == "body":
            continue 
          
        tissue = tissue.capitalize()
        tissue = rename_dict['tissues'].get(tissue, tissue)
                
        result.append({
            'tissue': tissue,
            'filename': full_atlas_data_folder / filename,
        })
    
    # assigning new value to result
    result = pd.DataFrame(result).set_index('tissue')
    # Order tissues alphabetically
    result = result.sort_index()['filename'].to_dict()
    return result

rename_dict = {
    'tissues': {
          'Body_wall': 'Skin',
          'Gut': 'Intestines',
          
         
    },
    'cell_types': {
        'sensory neuron': 'neuron',
        'epithelial cell': 'epithelial',
        'glial cell': 'glial',
        'muscle cell': 'muscle',
        'fat cell': 'adipocyte',
        'somatic precursor cell': 'stem cell',
        'gland': 'gland cell',
        'cardial cell': 'cardial',
        'tracheolar cell': 'tracheolar',
        'male germline cell': 'male germline',
        'female germline cell': 'female germline',
        'artefact': "",
        # TODO: split the various types in the following "systems"
        'excretory system': "",
        'male reproductive system': "",
        'female reproductive system': "",
    }
}

celltype_tissue_blacklist = {
}
      
coarse_cell_types = [
      
]


celltype_order = [
    ('immune', [
        'neuron',
        'hemocyte',
    ]),
    ('epithelial', [
        'epithelial',
    ]),
    ('endothelial', [
        
    ]),
    ('mesenchymal', [
        'adipocyte',
        'stem cell',
        'oenocyte',
    ]),
    ('other', [
        'muscle',
        'glial',
        'pigment cell',
        'gland cell',
        'tracheolar',
        'cardial',
        'male germline',
        'female germline',
        'ovary'
    ]),
]

if __name__ == '__main__':
#Remove existing compressed atlas file if present 
    if os.path.isfile(fn_out):
        os.remove(fn_out)

    compressed_atlas = {}

    tissue_sources = get_tissue_data_dict( 
        "d.melanogaster", full_atlas_data_folder, rename_dict)
    
    print(tissue_sources)
    tissues = list(tissue_sources.keys())
    
    tissues = [t for t in tissues if t != 'Head']
    tissue_sources = {key: value for key, value in tissue_sources.items() if key != "Head"}

#skip_tissues = ['Antenna', 'Brain', 'Gut', 'Haltere', 'Heart', 'Leg', 'Male_reproductive_glands', 'Malpighian_tubule', 'Oenocyte']

for it, (tissue, full_atlas_fn) in enumerate(tissue_sources.items()):

        #print(tissue)
        print(it+1, 'out of', len(tissues), (it + 1 * 100 // len(tissues)), '%', tissue)

        print('Load data for this tissue')
        adata_tissue = anndata.read(tissue_sources[tissue])
        
        #if tissue == "Head":
        #    continue
        #    #sc.pp.subsample(adata_tissue, fraction=0.1)

        #Renormalise to cptt
        print('Restart from raw data and renormalize')
        adata_tissue = adata_tissue.raw.to_adata()

        print('Data is logp1 of cptt, so undo the log bit')
        adata_tissue.X.data[:] = np.exp(adata_tissue.X.data) - 1

        print('Exclude cells that have inconsistencies in their annotation')
        #adata_tissue = adata_tissue[adata_tissue.obs['annotation_broad'] == adata_tissue.obs['R_annotation_broad']] (no longer needed?)
        if 'R_annoation_broad' in adata_tissue.obs:
                adata_tissue.obs['annotation_broad'] = adata_tissue.obs['R_annotation_broad'].astype(str)
        elif 'S_annotation_broad' in adata_tissue.obs:
                adata_tissue.obs['annotation_broad'] = adata_tissue.obs['S_annotation_broad'].astype(str)
        
        print('Exclude "unannotated"')
        adata_tissue = adata_tissue[adata_tissue.obs['annotation_broad'] != 'unannotated']
        
        print('Fix cell type annotations')
        adata_tissue.obs['cellType'] = fix_annotations(
                adata_tissue, 'annotation_broad', species, tissue,
                rename_dict, coarse_cell_types,
            )

        print(adata_tissue.obs['cellType'].value_counts())

        # Correction might declare some cells as untyped/low quality
        # they have an empty string instead of an actual annotation
        if (adata_tissue.obs['cellType'] == '').sum() > 0:
            idx = adata_tissue.obs['cellType'] != ''
            adata_tissue = adata_tissue[idx]

        celltypes = get_celltype_order(
            adata_tissue.obs['cellType'].value_counts().index,
            celltype_order,
        )
    
        print('Average')
        genes = adata_tissue.var_names
        avg_ge = pd.DataFrame(
                    np.zeros((len(genes), len(celltypes)), np.float32),
                    index=genes,
                    columns=celltypes,
                    )
        frac_ge = pd.DataFrame(
                    np.zeros((len(genes), len(celltypes)), np.float32),
                    index=genes,
                    columns=celltypes,
                    )
        ncells_ge = pd.Series(
                    np.zeros(len(celltypes), np.int64), index=celltypes,
                    )
        for celltype in celltypes:
            idx = adata_tissue.obs['cellType'] == celltype
            Xidx = adata_tissue[idx].X
            avg_ge[celltype] = np.asarray(Xidx.mean(axis=0))[0]
            frac_ge[celltype] = np.asarray((Xidx > 0).mean(axis=0))[0]
            ncells_ge[celltype] = idx.sum()

        compressed_atlas[tissue] = {
            "features": genes,
            "celltype": {
                "avg": avg_ge,
                "frac": frac_ge,
                "ncells": ncells_ge,
            },
        }

print('Consolidate gene list across tissues')
# Identify union of all gene lists via unique operations
genes = set()
for tissue, tdict in compressed_atlas.items():
    genest = list(tdict['features'])
    print(tissue, len(genest))
    genes = genes.union(set(genest))
genes = list(genes)

# Adapt the count tables for each tissue to use those genes, pad with zeros
for tissue, tdict in compressed_atlas.items():
    for table_name in ['avg', 'frac']:
        table = tdict['celltype'][table_name]
        genes_tissue = table.index
        # Create a new numerical table
        table_new = np.zeros(
            (len(genes), table.shape[1]),
            dtype=np.float32,
        )
        # Transform it into a data frame for convenience
        table_new = pd.DataFrame(
            table_new,
            index=genes,
            columns=table.columns,
        )
        table_new.index.name = "index"
        # Fill in the data from the old table
        table_new.loc[genes_tissue] = table.loc[genes_tissue]
        # Overwrite the old table for this tissue and table name (avg, frac)
        tdict['celltype'][table_name] = table_new
        tdict['features'] = pd.Index(genes)

print('Add gene annotations')
gene_annos = collect_gene_annotations(anno_fn, genes)
    
print('Store compressed atlas to file')
store_compressed_atlas(
            fn_out,
            compressed_atlas,
            tissues,
            None,
            celltype_order,
        )



{'Antenna': PosixPath('/Users/z5466829/cell_atlas_approximations_compression/data/full_atlases/drosophila_melanogaster/s_fca_biohub_antenna_10x.h5ad'), 'Haltere': PosixPath('/Users/z5466829/cell_atlas_approximations_compression/data/full_atlases/drosophila_melanogaster/s_fca_biohub_haltere_10x.h5ad'), 'Head': PosixPath('/Users/z5466829/cell_atlas_approximations_compression/data/full_atlases/drosophila_melanogaster/s_fca_biohub_head_10x.h5ad'), 'Heart': PosixPath('/Users/z5466829/cell_atlas_approximations_compression/data/full_atlases/drosophila_melanogaster/s_fca_biohub_heart_10x.h5ad'), 'Intestines': PosixPath('/Users/z5466829/cell_atlas_approximations_compression/data/full_atlases/drosophila_melanogaster/s_fca_biohub_gut_10x.h5ad'), 'Leg': PosixPath('/Users/z5466829/cell_atlas_approximations_compression/data/full_atlases/drosophila_melanogaster/s_fca_biohub_leg_10x.h5ad'), 'Male_reproductive_glands': PosixPath('/Users/z5466829/cell_atlas_approximations_compression/data/full_atlases/d

/var/folders/k8/9py9t33s5fs7pv2n0q4k25qr0000gq/T/ipykernel_63390/351612682.py:169: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_tissue.obs['cellType'] = fix_annotations(


cellType
neuron        12206
epithelial     8020
glial          2303
muscle          409
hemocyte         67
Name: count, dtype: int64
Average
2 out of 14 8 % Haltere
Load data for this tissue
Restart from raw data and renormalize
Data is logp1 of cptt, so undo the log bit
Exclude cells that have inconsistencies in their annotation
Exclude "unannotated"
Fix cell type annotations


/var/folders/k8/9py9t33s5fs7pv2n0q4k25qr0000gq/T/ipykernel_63390/351612682.py:169: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_tissue.obs['cellType'] = fix_annotations(


cellType
epithelial    3511
neuron         929
adipocyte      346
hemocyte       292
glial          164
muscle          39
Name: count, dtype: int64
Average
3 out of 14 9 % Heart
Load data for this tissue
Restart from raw data and renormalize
Data is logp1 of cptt, so undo the log bit
Exclude cells that have inconsistencies in their annotation
Exclude "unannotated"
Fix cell type annotations


...

13 out of 14 19 % Trachea
Load data for this tissue
Restart from raw data and renormalize
Data is logp1 of cptt, so undo the log bit
Exclude cells that have inconsistencies in their annotation
Exclude "unannotated"
Fix cell type annotations


/var/folders/k8/9py9t33s5fs7pv2n0q4k25qr0000gq/T/ipykernel_63390/351612682.py:169: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_tissue.obs['cellType'] = fix_annotations(


cellType
              10337
tracheolar     9508
neuron          502
muscle          218
adipocyte        61
Name: count, dtype: int64
Average
14 out of 14 20 % Wing
Load data for this tissue
Restart from raw data and renormalize
Data is logp1 of cptt, so undo the log bit
Exclude cells that have inconsistencies in their annotation
Exclude "unannotated"
Fix cell type annotations


/var/folders/k8/9py9t33s5fs7pv2n0q4k25qr0000gq/T/ipykernel_63390/351612682.py:169: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_tissue.obs['cellType'] = fix_annotations(


cellType
neuron      3415
glial        649
hemocyte     158
muscle       144
Name: count, dtype: int64
Average
Consolidate gene list across tissues
Antenna 13203
Haltere 12158
Heart 12674
Intestines 13407
Leg 11164
Male_reproductive_glands 13668
Malpighian_tubule 12843
Oenocyte 14644
Ovary 12460
Proboscis_and_maxillary_palps 12802
Skin 12924
Testis 15833
Trachea 14649
Wing 13411
Add gene annotations
Store compressed atlas to file


In [3]:
import h5py
h5_file = h5py.File("../data/atlas_approximations/d.melanogaster.h5")
h5_file.keys()

<KeysViewHDF5 ['gene_expression']>

In [23]:
h5_file['gene_expression']['by_tissue']['Antenna']['celltype']

array(['neuron', 'hemocyte', 'epithelial', 'muscle', 'glial'],
      dtype=object)

In [11]:
tdict

{'features': Index(['128up', '14-3-3epsilon', '14-3-3zeta', '140up', '18SrRNA-Psi:CR41602',
        '18w', '26-29-p', '28SrRNA-Psi:CR40596', '28SrRNA-Psi:CR40741',
        '28SrRNA-Psi:CR41609',
        ...
        'zf30C', 'zfh1', 'zfh2', 'zip', 'zld', 'zormin', 'zpg', 'zuc', 'zyd',
        'zye'],
       dtype='object', name='index', length=13411),
 'celltype': {'avg':                   neuron  hemocyte    muscle     glial
  index                                                 
  cbc             0.063151  0.042764  0.023988  0.065609
  kn              0.217027  0.105310  0.000000  0.021332
  fidipidine      0.116487  0.058817  0.047500  0.086695
  CG33758         0.000000  0.000000  0.000000  0.000000
  lncRNA:CR45127  0.020842  0.013741  0.000000  0.002200
  ...                  ...       ...       ...       ...
  Caf1-105        0.045015  0.000000  0.047500  0.045786
  CG10853         0.000000  0.000000  0.000000  0.000000
  CG12609         0.000000  0.000000  0.000000  0.000000
 

In [44]:
print('Consolidate gene list across tissues')

# Identify union of all gene lists via unique operations
genes = set()
for tissue, tdict in compressed_atlas.items():
    genest = list(tdict['features'])
    print(tissue, len(genest))
    genes = genes.union(set(genest))
genes = list(genes)

# Adapt the count tables for each tissue to use those genes, pad with zeros
for tissue, tdict in compressed_atlas.items():
    for table_name in ['avg', 'frac']:
        table = tdict['celltype'][table_name]
        genes_tissue = table.index
        # Create a new numerical table
        table_new = np.zeros(
            (len(genes), table.shape[1]),
            dtype=np.float32,
        )
        # Transform it into a data frame for convenience
        table_new = pd.DataFrame(
            table_new,
            index=genes,
            columns=table.columns,
        )
        table_new.index.name = "index"
        # Fill in the data from the old table
        table_new.loc[genes_tissue] = table.loc[genes_tissue]
        # Overwrite the old table for this tissue and table name (avg, frac)
        tdict['celltype'][table_name] = table_new

Consolidate gene list across tissues
Antenna 13203
Haltere 12158
Heart 12674
Intestines 13407
Leg 11164
Male_reproductive_glands 13668
Malpighian_tubule 12843
Oenocyte 14644


In [7]:
sc.pp.subsample?

In [45]:
tdict

{'features': Index(['128up', '14-3-3epsilon', '14-3-3zeta', '140up', '18SrRNA-Psi:CR41602',
        '18w', '26-29-p', '28SrRNA-Psi:CR40596', '28SrRNA-Psi:CR40741',
        '28SrRNA-Psi:CR41609',
        ...
        'zf30C', 'zfh1', 'zfh2', 'zip', 'zld', 'zormin', 'zpg', 'zuc', 'zyd',
        'zye'],
       dtype='object', name='index', length=14644),
 'celltype': {'avg':                    neuron  hemocyte  epithelial  adipocyte  oenocyte  \
  index                                                                  
  CG9902           0.274321  0.015832    0.010117   0.007317  0.079422   
  Su(fu)           0.057585  0.021780    0.118406   0.052984  0.085729   
  Sfp38D           0.000000  0.000000    0.010722   0.007831  0.006396   
  lncRNA:CR44553   0.000000  0.086571    0.027289   0.000231  1.955495   
  AGBE             0.200099  0.502133    0.457042   2.257712  0.801543   
  ...                   ...       ...         ...        ...       ...   
  mirr             2.744868  0.81937

In [18]:
it, (tissue, data) = [0, ['lung', 999887]]

In [25]:
{'name': 'Joanna', 'role': 'data scientist'}

dict(name='Joanna', role='data scientist', timeinoffice=3, ncolleagues=5)

{'name': 'Joanna',
 'role': 'data scientist',
 'timeinoffice': 3,
 'ncolleagues': 5}

In [ ]:
a = {
    'name': 'Joanna',
    'role': 'data scientist',
}
{ name, role } = a

name = a['name']
role = a['role']

In [ ]:
tissue_specific = "Gut"
tissue_specific_obs = adata_tissue.obs[adata_tissue.obs['tissue'] == tissue_specific]
print(tissue_specific_obs.columns)

Index(['age', 'batch', 'batch_id', 'celda_decontx__clusters',
       'celda_decontx__contamination',
       'celda_decontx__doublemad_predicted_outliers', 'dissection_lab',
       'fca_id', 'fly_genetics', 'id', 'log_n_counts', 'log_n_genes',
       'n_counts', 'n_genes', 'note', 'percent_mito', 'sample_id',
       'scrublet__doublet_scores', 'scrublet__predicted_doublets',
       'scrublet__predicted_doublets_based_on_10x_chromium_spec', 'sex',
       'tissue', 'leiden_res0.4', 'leiden_res0.6', 'leiden_res0.8',
       'leiden_res1.0', 'leiden_res1.2', 'leiden_res1.4', 'leiden_res1.6',
       'leiden_res1.8', 'leiden_res10.0', 'leiden_res2.0', 'leiden_res4.0',
       'leiden_res6.0', 'leiden_res8.0', 'annotation',
       'annotation__ontology_id', 'annotation_broad',
       'annotation_broad__ontology_id', 'annotation_broad_extrapolated',
       'annotation_broad_extrapolated__ontology_id', 'R_annotation',
       'R_annotation__ontology_id', 'R_annotation_broad',
       'R_annotation_b

In [ ]:
table_of_contents = tissue_specific_obs.columns
print(table_of_contents)


Index(['age', 'batch', 'batch_id', 'celda_decontx__clusters',
       'celda_decontx__contamination',
       'celda_decontx__doublemad_predicted_outliers', 'dissection_lab',
       'fca_id', 'fly_genetics', 'id', 'log_n_counts', 'log_n_genes',
       'n_counts', 'n_genes', 'note', 'percent_mito', 'sample_id',
       'scrublet__doublet_scores', 'scrublet__predicted_doublets',
       'scrublet__predicted_doublets_based_on_10x_chromium_spec', 'sex',
       'tissue', 'leiden_res0.4', 'leiden_res0.6', 'leiden_res0.8',
       'leiden_res1.0', 'leiden_res1.2', 'leiden_res1.4', 'leiden_res1.6',
       'leiden_res1.8', 'leiden_res10.0', 'leiden_res2.0', 'leiden_res4.0',
       'leiden_res6.0', 'leiden_res8.0', 'annotation',
       'annotation__ontology_id', 'annotation_broad',
       'annotation_broad__ontology_id', 'annotation_broad_extrapolated',
       'annotation_broad_extrapolated__ontology_id', 'R_annotation',
       'R_annotation__ontology_id', 'R_annotation_broad',
       'R_annotation_b